<a href="https://colab.research.google.com/github/Lucas-Ellenberger/CSE144/blob/main/stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os

In [ ]:
# make sure you're logged in with `huggingface-cli login`
from huggingface_hub import login

login()

In [ ]:
!pip install torch
!pip install diffusers
!pip install accelerate
!pip install datasets
!pip install datasets[vision]

In [ ]:
import torch
from torch import autocast
import accelerate
from diffusers import StableDiffusionPipeline

In [ ]:
from diffusers import DDPMPipeline
from accelerate import Accelerator
from datasets import load_dataset

dataset = load_dataset("AI4Math/MathVista")

In [ ]:
# print the first example on the testmini set
print(dataset["testmini"][0])
print(dataset["testmini"][0]['pid']) # print the problem id
print(dataset["testmini"][0]['question']) # print the question text
print(dataset["testmini"][0]['query']) # print the query text
print(dataset["testmini"][0]['image']) # print the image path
print(dataset["testmini"][0]['answer']) # print the answer
dataset["testmini"][0]['decoded_image'] # display the image

# print the first example on the test set
print(dataset["test"][0])

In [ ]:
# Uncomment if you want to see a sample image from the dataset
# dataset["testmini"][0]['decoded_image']

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
pipe = StableDiffusionPipeline.from_pretrained(
	"CompVis/stable-diffusion-v1-4",
	use_auth_token=True
).to(device)

prompt = "a photo of an astronaut riding a horse on mars"
with autocast(device):
    image = pipe(prompt)["sample"][0]

image.save("astronaut_rides_horse.png")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: User specified an unsupported autocast device_type 'cpu'